### Import Libraries

In [1]:
import pandas as pd # Dataset operations
import numpy as np # math and array operations
from scipy.spatial import distance

### Import Dataset
Import dataset with the following columns:
- Propierties
- Units
- Max or Min Criteria
- Next columns are for each material with its properties values

In [2]:
df = pd.read_excel("Materiales.xlsx")
df.head()

,Propiedad,Unidades,Criterio,PP,LDPE,HDPE,N6
0,Esfuerzo fluencia,Mpa,Max,41.40,78.000,38.00,79.00
1,Módulo de elasticidad,Gpa,Max,1.77,0.380,1.50,2.90
2,Densidad,g/cm^3,Min,0.92,0.925,0.96,1.14
3,Absorción de agua,%,Min,0.02,0.015,0.20,1.80
4,Costo de la materia prima,USD/kg,Min,0.97,1.060,0.90,2.10


In [3]:
# List of criterias
criterias = df["Propiedad"].tolist()
criterias

['Esfuerzo fluencia',
 'Módulo de elasticidad',
 'Densidad',
 'Absorción de agua',
 'Costo de la materia prima']

In [4]:
# List of alternatives
alternatives =  df.columns.tolist()[3:]
alternatives

['PP', 'LDPE', 'HDPE', 'N6']

In [5]:
# Max or min criterias
maxmin = df["Criterio"].tolist()
maxmin

['Max', 'Max', 'Min', 'Min', 'Min']

# VIKOR

### Decisional matrix

In [6]:
df.pop("Unidades") # Deleta Units column
df.pop("Criterio") # Delete max-min criteria colum
df.pop("Propiedad")# Delete properties column
df

,PP,LDPE,HDPE,N6
0,41.40,78.000,38.00,79.00
1,1.77,0.380,1.50,2.90
2,0.92,0.925,0.96,1.14
3,0.02,0.015,0.20,1.80
4,0.97,1.060,0.90,2.10


In [7]:
df_decisional = df.T
df_decisional.columns = criterias
df_decisional

,Esfuerzo fluencia,Módulo de elasticidad,Densidad,Absorción de agua,Costo de la materia prima
PP,41.4,1.77,0.920,0.020,0.97
LDPE,78.0,0.38,0.925,0.015,1.06
HDPE,38.0,1.50,0.960,0.200,0.90
N6,79.0,2.90,1.140,1.800,2.10


In [8]:
np.set_printoptions(suppress=True, precision=5) # Avoid scientific notation in numpy matrix and 3 decimal digits

m_decisional = df_decisional.to_numpy() #Dataframe into numpy matrix
m_decisional

array([[41.4  ,  1.77 ,  0.92 ,  0.02 ,  0.97 ],
       [78.   ,  0.38 ,  0.925,  0.015,  1.06 ],
       [38.   ,  1.5  ,  0.96 ,  0.2  ,  0.9  ],
       [79.   ,  2.9  ,  1.14 ,  1.8  ,  2.1  ]])

### Best and Worst Performance

In [9]:
f_max = np.zeros(len(criterias)) # Arrays to save the best performance alternative
f_min = np.zeros(len(criterias)) # Arrays to save the worst performance alternative

for i in range(len(criterias)):
    
    if maxmin[i] == "Max":
        f_max[i] = max(m_decisional[:,i])
        f_min[i] = min(m_decisional[:,i])
    
    elif maxmin[i] == "Min":
        f_max[i] = min(m_decisional[:,i])
        f_min[i] = max(m_decisional[:,i])
        
print("f+")
print(f_max)
print(" ")
print("f-")
print(f_min)

f+
[79.     2.9    0.92   0.015  0.9  ]
 
f-
[38.    0.38  1.14  1.8   2.1 ]


### Weights

In [10]:
w = np.zeros(len(criterias)) # Array to save criterias' weight

condition = True # Cndition sum weights = 1
 
while condition:
    
    for i in range(len(criterias)):
        w[i] = float(input("Weight of {}: ".format(criterias[i])))
    
    if np.sum(w) == 1:
        condition = False
    else:
        print(" ")
        print("¡Sum of weights is different to 1, correct the values!")

print(" ")
list(zip(criterias,w))

Weight of Esfuerzo fluencia: 0.1
Weight of Módulo de elasticidad: 0.15
Weight of Densidad: 0.3
Weight of Absorción de agua: 0.1
Weight of Costo de la materia prima: 0.35
 


[('Esfuerzo fluencia', 0.1),
 ('Módulo de elasticidad', 0.15),
 ('Densidad', 0.3),
 ('Absorción de agua', 0.1),
 ('Costo de la materia prima', 0.35)]

### S-R matrix

In [11]:
m_SR = np.zeros((len(alternatives),len(criterias))) # Matrix to save SR values

for i in range(len(alternatives)):
    
    m_SR[i,:] = (f_max-m_decisional[i,:])/(f_max-f_min)*w

print(m_SR)

[[ 0.09171  0.06726 -0.       0.00028  0.02042]
 [ 0.00244  0.15     0.00682 -0.       0.04667]
 [ 0.1      0.08333  0.05455  0.01036 -0.     ]
 [ 0.       0.       0.3      0.1      0.35   ]]


### S & R distances

In [12]:
S = np.zeros(len(alternatives)) # Manhathan Distance
R = np.zeros(len(alternatives)) # Chebyshev Disntace

for i in range(len(alternatives)):
    
    S[i] = sum(m_SR[i,:])
    R[i] = max(m_SR[i,:])
    
print("S")
print(S)
print(" ")
print("R")
print(R)

S
[0.17967 0.20592 0.24824 0.75   ]
 
R
[0.09171 0.15    0.1     0.35   ]


### S *, R *, S-, R-

In [13]:
S1 = min(S) 
R1 = min(R)

S2 = max(S)
R2 = max(R)

print("S*: {:.4}".format(S1))
print("R*: {:.4}".format(R1))
print("S-: {:.4}".format(S2))
print("R-: {:.4}".format(R2))

S*: 0.1797
R*: 0.09171
S-: 0.75
R-: 0.35


### Merit Function

In [14]:
v = float(input("Input the strategy weight: "))

Q = v*(S-S1)/(S2-S1) + (1-v)*(R-R1)/(R2-R1)

print(Q)

Input the strategy weight: 0.5
[0.      0.13586 0.07617 1.     ]


### Results

In [15]:
# Show it as a Dataframe
df_end = pd.DataFrame(Q)
df_end.insert(0, "Material", alternatives)
df_end.columns = ["Material","Merit Function"]
df_end

,Material,Merit Function
0,PP,0.000000
1,LDPE,0.135862
2,HDPE,0.076173
3,N6,1.000000


In [16]:
df_end = df_end.sort_values(by=["Merit Function"], ascending = True)
df_end.insert(0, "Ranking", list(range(1,len(alternatives)+1)))
df_end

,Ranking,Material,Merit Function
0,1,PP,0.000000
2,2,HDPE,0.076173
1,3,LDPE,0.135862
3,4,N6,1.000000
